In [19]:
# call documents
from langchain_community.document_loaders import PyPDFLoader
from pprint import pprint
loader = PyPDFLoader("https://arxiv.org/pdf/2310.05421")
document = loader.load()
print(len(document))

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200, separators=["\n","\n\n"])
chunks = text_splitter.split_documents(document)

texts = [chunk.page_content for chunk in chunks]

pprint(f"Number of chunks: {len(chunks)}")
pprint(f"First chunk: {texts[0]}")

4
'Number of chunks: 10'
('First chunk: Submitted to the 3rd International Conference on “Women in '
 'Science & Technology: Creating Sustainable Career”  \n'
 '28 -30 December, 2023 \n'
 'Automating Customer Service using LangChain \n'
 'Building custom open-source GPT Chatbot for organizations \n'
 'Keivalya Pandya \n'
 '19me439@bvmengineering.ac.in \n'
 'Birla Vishvakarma Mahavidyalaya, Gujarat, India \n'
 'Prof. Dr. Mehfuza Holia \n'
 'msholia@bvmengineering.ac.in \n'
 'Birla Vishvakarma Mahavidyalaya, Gujarat, India \n'
 ' \n'
 'Abstract— In the digital age, the dynamics of customer \n'
 'service are evolving, driven by technological advancements and \n'
 'the integration of Large Language Models (LLMs). This research \n'
 'paper introduces a groundbreaking approach to automating \n'
 'customer service using LangChain, a custom LLM tailored for \n'
 'organizations. The paper explores the obsolescence of traditional \n'
 'customer support techniques, particul arly Frequently Asked 

In [ ]:
[print(chunk.page_content,"end\n") for chunk in chunks]

`Embedding models`

Embedding models are specifically designed to interface with text embeddings.
Embeddings generate a vector representation for a specified piece or "chunk" of text.  Embeddings offer the advantage of allowing you to conceptualize text within a vector space. Consequently, you can perform operations such as semantic search, where you identify pieces of text that are most similar within the vector space.


In [4]:
import ollama

# Example: Generate embeddings for a text using the model
text = "Hello, world!"
embeddings = ollama.embeddings(model='qwen3-embedding:0.6b', prompt=text)
print(embeddings)

embedding=[0.00622900202870369, 0.009505381807684898, -0.00968700461089611, -0.06248854100704193, 0.007647708058357239, 0.00596900237724185, -0.01175885833799839, 0.005585320293903351, -0.10474134981632233, -0.024318154901266098, -0.018558606505393982, -0.023536421358585358, 0.02585569955408573, -0.008519202470779419, -0.04043920710682869, 0.0953800156712532, -0.0002125309983966872, 0.08306290209293365, 0.10666666179895401, -0.06285212188959122, -0.013290720991790295, 0.03755147010087967, -0.030469710007309914, 0.09925328195095062, -0.03276112675666809, -0.026558518409729004, -0.044985581189394, 0.10583758354187012, 0.0017894285265356302, -0.01217898540198803, 0.0013039986370131373, 0.0733412578701973, -0.026383496820926666, -0.014265338890254498, -0.04156574606895447, -0.013515258207917213, 0.0280031505972147, 0.013641751371324062, -0.0213386882096529, 0.05428208038210869, -0.015943247824907303, 0.0021830855403095484, 0.043972499668598175, -0.014688925817608833, 0.028029097244143486, 

In [5]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.vectorstores import InMemoryVectorStore
import numpy as np

embedding_model = OllamaEmbeddings(model='qwen3-embedding:0.6b')

In [ ]:

embedding_result = embedding_model.embed_documents(texts)
pprint(f"shape of embedding result: {np.array(embedding_result).shape}")

In [ ]:
pprint(f"embedding first 5 values: {embedding_result[0][0:5]}", width=120)
pprint(f"shape of first embedding: {np.array(embedding_result[0]).shape}")

#### Vector stores

One of the most common ways to store and search over unstructured data is to embed the text data and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. You can use a [vector store](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/) to store embedded data and perform vector search for you.

Next, have the embedding model perform the embedding process and store the resulting vectors in the Chroma vector database.

**NOTE**: You can safely ignore the warnings related to telemetry events. They are related to ChromaDB's telemetry collection system and do not affect the functionality of your code. Your vector search and similarity operations will work correctly despite these messages.



In [ ]:
docsearch = Chroma.from_documents(documents=chunks, 
                                  embedding=embedding_model)

query = "Abstract"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

#### Retrievers

A retriever is an interface that returns documents using an unstructured query. Retrievers are more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. You can still use vector stores as the backbone of a retriever. Note that other types of retrievers also exist.

Retrievers accept a string `query` as input and return a list of `Documents` as output.

You can view a list of the advanced retrieval types LangChain supports at [https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/)

##### **Vector store-backed retrievers**
Vector store retrievers are retrievers that use a vector store to retrieve documents. They are a lightweight wrapper around the vector store class to make it conform to the retriever interface. They use the search methods implemented by a vector store, such as similarity search and MMR (Maximum marginal relevance), to query the texts in the vector store.

Now that you have constructed a vector store `docsearch`, you can easily construct a retriever such as seen in the following code.



In [ ]:
retriever = docsearch.as_retriever()
doc = retriever.invoke(query)
pprint(doc[0] , width=20)

In [ ]:
def search_documents(query, top_k=3):
    """Search for documents relevant to a query"""
    # Use the retriever to get relevant documents
    docs = retriever._get_relevant_documents(query , run_manager=None)
    
    # Limit to top_k if specified
    return docs[:top_k]

test_queries = [
    "What is LangChain?",
    "How do retrievers work?",
    "Why is document splitting important?"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = search_documents(query)
    print(f"Amount of source:{len(results)}")
    for i in results:
        print(i.page_content,"\n")

In [7]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model='gpt-oss:20b-cloud',
                max_tokens=512,
                temperature=0.2,
                top_p=0.9,)

# ฟังก์ชันสำหรับรวมเนื้อหาเพื่อส่งให้ LLM
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel


def RAG_answer_the_questions(query: str ,
                             retriever:VectorStoreRetriever , 
                             llm:OllamaLLM):

    system_prompt = """คุณเป็นผู้ช่วยผู้เชี่ยวชาญด้านงานวิจัย ตอบคำถามโดยใช้ข้อมูลจาก Context ที่ให้มาเท่านั้น
                        ### Context:
                        {context} 
                        ### คำแนะนำในการตอบ:
                            - ทำความเข้าใจเกี่ยวกับคำถามและ Context อย่างละเอียดโดยคิดและแปลเป็นภาษาไทยก่อน ไม่สนใจพิมเล็กพิมพ์ใหญ่
                            - ทำความเข้าใจว่า user อาจจะต้องสะกดคำถามผิด
                            - ให้ดูว่าเป็นไปได้ไหมที่คำถามที่จะเกี่ยวข้องกับ Context
                            - หากไม่เกี่ยวกับ Context ให้ตอบให้สอดคล้องกับ Context และอย่าเดาคำตอบ
                            - หากไม่เกี่ยวข้องกับ context ให้ตอบว่า "อาจจะต้องกาถามว่า...รึเปล่า"
                            - ตอบในรูปแบบที่กระชับและเข้าใจง่าย
                    """

    user_prompt = """
                        ### คำถาม: {input}
                        คำตอบ:
                    """

    prompt = ChatPromptTemplate.from_messages([
                                            ("system", system_prompt), 
                                            ("human", user_prompt)
                                            ])
    try:
        # สร้างส่วนการทำงานหลัก
        rag_chain_from_docs = (
            RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])) )
            | prompt
            | llm
            | StrOutputParser()
        )
    except Exception as e:
        raise print(f"Error creating RAG chain: {e}")
   
    # รวมร่างเข้ากับ Retriever
    full_rag_chain = RunnableParallel(
        {"context": retriever, "input": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)

    result = full_rag_chain.invoke(query)

    return result

In [ ]:
# สร้าง Chain แบบแยกส่วนเพื่อให้คืนค่าทั้งเอกสารและคำตอบ
retriever = docsearch.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 8, 'fetch_k': 20, 'lambda_mult': 0.5}
)

# --- วิธีการเรียกใช้งาน ---
result = RAG_answer_the_questions(query="สรุปแนวคิดหลักของ LangChain และอธิบายการทำงานของ Retriever ในระบบ RAG",
                                  retriever=retriever,
                                 llm=llm)
print(f"--- ผลลัพธ์ RAG ---\n{result}")
# 1. ดูคำตอบ
print(f"คำตอบ: {result['answer']}")

# 2. ดู Metadata (เช่น ชื่อไฟล์ หรือ หน้าที่อ้างอิง)
print("\n--- แหล่งที่มา (Metadata) ---")
for i, doc in enumerate(result['context']):
    print(f"เอกสารที่ {i+1}:")
    print(f"เนื้อหาบางส่วน: {doc.page_content[:100]}...")
    print(f"Metadata: {doc.metadata}") # นี่คือจุดที่ดู Metadata

In [ ]:
parameters = {"query": "ผู้เขียนรายงานดำเนินการศึกอย่างไร ขอสรุปเป็นขั้นตอนทีละขั้น",
              "retriever": retriever,
              "llm": llm }

def result_question_RAG(result: dict):
    
    print(f"คำตอบ: {result['answer']}")

    # 2. ดู Metadata (เช่น ชื่อไฟล์ หรือ หน้าที่อ้างอิง)
    print("\n--- แหล่งที่มา (Metadata) ---")
    for i, doc in enumerate(result['context']):
        print(f"เอกสารที่ {i+1}:")
        print(f"เนื้อหาบางส่วน: {doc.page_content[:100]}...")
        print(f"Metadata: {doc.metadata}") # นี่คือจุดที่ดู Metadata

result_question_RAG(result = RAG_answer_the_questions(**parameters))

In [20]:
from typing import List
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.stores import InMemoryStore



# 1. สร้าง Class สำหรับ Parent Retriever เอง (เพื่อแก้ปัญหา ModuleNotFoundError)
class MyParentRetriever(BaseRetriever):
    vectorstore: any
    docstore: any = InMemoryStore()
    id_key: str = "doc_id"

    k: int = 10
    fetch_k: int = 20 # ค้นหาเบื้องต้น 20 ชิ้น
    lambda_mult: float = 0.8

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1. หาชิ้นลูกที่ใกล้เคียงที่สุด
        child_docs = self.vectorstore.max_marginal_relevance_search(query, 
            k=self.k, 
            fetch_k=self.fetch_k, 
            lambda_mult=self.lambda_mult)
        
        # 2. รวบรวม ID ของแม่จาก metadata ของลูก
        parent_ids = []
        for child in child_docs:
            pid = child.metadata.get(self.id_key)
            if pid:
                parent_ids.append(pid)
        
        # 3. ลบ ID ที่ซ้ำกันออก
        unique_parent_ids = list(dict.fromkeys(parent_ids))
        
        # 4. ไปดึง "ชิ้นแม่" ตัวจริงจาก docstore
        parent_docs = self.docstore.mget(unique_parent_ids)
        
        # กรองค่า None ออก (กรณีหาใน docstore ไม่เจอ)
        return [d for d in parent_docs if d is not None]


vectorstore = Chroma(collection_name="research_db", embedding_function=embedding_model)
id_key = "doc_id"

# สร้างตัวแปร Retriever
retriever = MyParentRetriever(vectorstore=vectorstore, id_key=id_key)

# 3. สร้างตัวช่วยตัดแบ่งข้อความ (Splitters)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)

from uuid import uuid4

def add_documents_to_system(retriever, docs):
    # ล้างข้อมูลเก่าก่อนเพื่อความชัวร์ (ถ้าใช้ InMemory)
    # retriever.vectorstore = InMemoryVectorStore(embeddings) 
    # retriever.docstore = InMemoryStore()

    total_parents = 0
    for doc in docs:
        # 1. ตัดชิ้นแม่ (Parent)
        parents = parent_splitter.split_documents([doc])
        
        for p in parents:
            # สร้าง ID เฉพาะสำหรับชิ้นแม่ชิ้นนี้
            parent_id = str(uuid4())
            p.metadata[retriever.id_key] = parent_id
            
            # 2. ตัดชิ้นลูก (Child) จากแม่ชิ้นนี้
            children = child_splitter.split_documents([p])
            for c in children:
                c.metadata[retriever.id_key] = parent_id
            
            # 3. บันทึกลูกลง Vectorstore
            retriever.vectorstore.add_documents(children)
            
            # 4. บันทึกแม่ลง Docstore (ต้องส่งเป็น list ของ tuple: [(key, value)])
            # ตรวจสอบให้แน่ใจว่า retriever.docstore คือตัวเดียวกับที่คุณใช้ค้นหา
            retriever.docstore.mset([(parent_id, p)])
            total_parents += 1
            
    print(f"✅ เพิ่มสำเร็จ! จำนวน Parent ใน Store: {len(list(retriever.docstore.yield_keys()))} ชิ้น")



add_documents_to_system(retriever, document)


✅ เพิ่มสำเร็จ! จำนวน Parent ใน Store: 10 ชิ้น


In [21]:
def debug_retriever(query, retriever):
    # 1. ลองหาในระดับ Child ก่อน (Vector Store)
    child_results = retriever.vectorstore.similarity_search(query, k=1)
    if not child_results:
        print("❌ แม้แต่ชิ้น 'ลูก' ก็ยังหาไม่เจอ: ตรวจสอบว่ามีการ add_documents หรือยัง?")
        return

    child_doc = child_results[0]
    pid = child_doc.metadata.get(retriever.id_key)
    print(f"✅ เจอชิ้นลูก! Content: {child_doc.page_content[:50]}...")
    print(f"🔗 ID ที่อ้างอิงไปหาชิ้นแม่: {pid}")

    # 2. ลองหาในระดับ Parent (Docstore)
    if pid:
        parent_doc = retriever.docstore.mget([pid])
        if parent_doc and parent_doc[0]:
            print(f"✅ เจอชิ้นแม่! ความยาว: {len(parent_doc[0].page_content)}")
        else:
            print("❌ พบ ID ในลูก แต่หาเนื้อหาใน Docstore (ชิ้นแม่) ไม่เจอ!")
    else:
        print("❌ ชิ้นลูกที่หาเจอ ไม่มี Metadata ID ติดมาด้วย")

# เรียกใช้เพื่อหาสาเหตุ
debug_retriever("chain", retriever)

✅ เจอชิ้นลูก! Content: harnessed to create effici ent, personalized, and ...
🔗 ID ที่อ้างอิงไปหาชิ้นแม่: 057c2bce-ffdb-4a76-87f3-7493c1864b72
✅ เจอชิ้นแม่! ความยาว: 1980


In [ ]:
test_docs = retriever.invoke("""chain""")

if test_docs:
    print(f"ยืนยันความยาว Parent: {len(test_docs[0].page_content)}")
    print(test_docs)
else:
    print("ดึงข้อมูลไม่สำเร็จ")

In [8]:
from langchain_community.retrievers import ArxivRetriever

retriever = ArxivRetriever(
    load_max_docs=4,
    get_full_documents=True,
    doc_content_chars_max=None
)
document = retriever.invoke("2310.05421")
docsearch = Chroma.from_documents(documents=document, 
                                  embedding=embedding_model)

In [9]:
from operator import itemgetter
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory,RunnablePassthrough
from langchain_core.prompts import  MessagesPlaceholder,ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel


# 1. สร้างที่เก็บ History (แทนที่ ConversationBufferMemory)
# ในการใช้งานจริงคุณอาจใช้ RedisChatMessageHistory หรือ Postgres แทนได้
# store = {}
retriever = docsearch.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 10, 'fetch_k': 20, 'lambda_mult': 0.95}
)
def format_docs(docs):
    if not docs:
        return ""
    # กรองเอาเฉพาะ doc ที่ไม่ใช่ None ออกมา
    valid_docs = [doc for doc in docs if doc is not None]
    if not valid_docs:
        return "ไม่พบเนื้อหาที่เกี่ยวข้องในฐานข้อมูล"
    return "\n\n".join(doc.page_content for doc in valid_docs)
                       
chat_history_db = {}
def get_session_history(session_id: str):
    if session_id not in chat_history_db:
        chat_history_db[session_id] = InMemoryChatMessageHistory()
    return  chat_history_db[session_id]

def RAG_answer_the_questions_history(query: str ,
                             retriever:any , 
                             llm:any,
                             session_history:dict = get_session_history,**kwargs ):

    system_prompt = """คุณคือผู้ช่วยวิจัยอัจฉริยะที่มีทักษะในการวิเคราะห์เอกสารวิชาการ 
                    หน้าที่ของคุณคือตอบคำถามโดยใช้ "Context" (เนื้อหาจากงานวิจัย) ที่เตรียมไว้ให้ด้านล่างนี้เท่านั้น

                        กฎข้อบังคับในการตอบ:
                        1. **ยึดตามหลักฐาน:** ตอบคำถามโดยใช้ข้อมูลที่ปรากฏใน Context เท่านั้น หากใน Context ไม่มีคำตอบ ให้ตอบตรงๆ ว่า "ขออภัยครับ ข้อมูลที่ระบุในงานวิจัยส่วนนี้ไม่เพียงพอที่จะตอบคำถามได้" ห้ามแต่งเนื้อหาขึ้นมาเอง (Hallucination)
                        2. **การอ้างอิง:** หากเป็นไปได้ ให้ระบุส่วนของงานวิจัยที่นำมาตอบ เช่น (จากส่วนบทนำ) หรือ (จากผลการทดลองในตารางที่ 1)
                        3. **ความเป็นกลาง:** ใช้ภาษาที่เป็นทางการ เป็นกลาง และกระชับ
                        4. **โครงสร้าง:** หากคำตอบมีหลายประเด็น ให้ใช้ Bullet points เพื่อความชัดเจน
                        5. **ความไม่แน่ชัด:** หากข้อมูลในงานวิจัยมีความกำกวม ให้ระบุว่า "งานวิจัยระบุว่า... แต่ไม่ได้ขยายความในประเด็น..."

                        Context:
                        {context}
                            
                    """

    user_prompt = """
                        ### Question: {input}
                        คำตอบ:
                    """

    prompt = ChatPromptTemplate.from_messages([
                                            ("system", system_prompt),
                                            MessagesPlaceholder(variable_name="chat_history"), 
                                            ("human", user_prompt)
                                            ])
    try:
        # สร้างส่วนการทำงานหลัก
        rag_chain_from_docs = (
            RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])) )
            | prompt
            | llm
            | StrOutputParser()
        )
    except Exception as e:
        raise print(f"Error creating RAG chain: {e}")
   

    full_rag_chain = RunnableParallel(
        {"context": itemgetter("input")|retriever, 
         "input": itemgetter("input"),
        "chat_history": itemgetter("chat_history")}
    ).assign(answer=rag_chain_from_docs)

    
    # 2. ปรับ Chain เดิมของคุณให้รองรับ History
    # สมมติว่าคุณมี rag_chain จากข้อที่แล้ว
    with_message_history = RunnableWithMessageHistory(
        full_rag_chain,
        session_history,
        input_messages_key="input",
        history_messages_key="chat_history", # ชื่อเดียวกับที่คุณเคยใช้ใน prompt
        output_messages_key="answer",
    )

    # 3. การเรียกใช้งาน (ต้องส่ง session_id)
    config = {"configurable": {"session_id": kwargs.get("session_id") }}
    response = with_message_history.invoke(
        {"input": query},
        config=config
    )

    return response


In [10]:
parameters = {"query": "ผู้เขียนรายงานดำเนินการศึกอย่างไร  ขอสรุปเป็นขั้นตอนทีละขั้น",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_123" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

**ขั้นตอนการดำเนินการของผู้เขียน (สรุปตามเนื้อหาในงานวิจัย)**  

- **1. กำหนดวัตถุประสงค์และบริบท**  
  - สร้าง chatbot “Sahaay” ที่ใช้ LangChain เพื่อให้บริการลูกค้าแบบอัตโนมัติ  
  - ใช้ข้อมูลจากเว็บไซต์ของ Birla Vishvakarma Mahavidyalaya (BVM) เป็นฐานความรู้ (จากบทนำ)

- **2. เก็บรวบรวมข้อมูล**  
  - ใช้ BeautifulSoup ในการ web‑scrape ข้อมูลจากหน้าเว็บหลักของ BVM (FAQs, คู่มือ, ฟอรั่ม ฯลฯ)  
  - เก็บข้อมูลเป็นชุดข้อความที่ใช้เป็น “context” ของโมเดล (จากส่วน Methodology A)

- **3. สร้างและจัดเก็บ embeddings**  
  - ใช้โมเดล “hkunlp/instructor‑large” ของ HuggingFace เพื่อสร้างเวกเตอร์ embedding ของข้อความ  
  - เก็บ embeddings ใน FAISS เพื่อให้ค้นหาแบบ similarity‑search ได้รวดเร็ว (จากส่วน Methodology B)

- **4. เลือกและฝึกโมเดลภาษา**  
  - เปรียบเทียบ Flan‑T5 รุ่นต่าง ๆ (XXL, BASE, SMALL)  
  - เลือก Flan‑T5‑XXL เป็นโมเดลหลัก เนื่องจากให้ผลตอบสนองดีที่สุด (จากส่วน Methodology C)

- **5. ผสาน LangChain กับโมเดลและ embeddings**  
  - สร้าง LangChain agent ที่ใช้ embeddings จาก FAISS เพ

In [12]:
parameters = {"query": "สรุปประเด็นสำคัญคำตอบก่อนหน้า",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_123" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

**สรุปประเด็นสำคัญจากขั้นตอนการดำเนินงาน**

- **วัตถุประสงค์**: สร้าง chatbot “Sahaay” ด้วย LangChain เพื่อให้บริการลูกค้าอัตโนมัติ โดยใช้ข้อมูลจากเว็บไซต์ BVM เป็นฐานความรู้ (จากบทนำ)  
- **การรวบรวมข้อมูล**: ใช้ BeautifulSoup สแกนเว็บ BVM เพื่อเก็บ FAQs, คู่มือ, ฟอรั่ม ฯลฯ (จากส่วน Methodology A)  
- **สร้าง embeddings**: ใช้โมเดล “hkunlp/instructor‑large” สร้างเวกเตอร์และเก็บใน FAISS เพื่อค้นหาแบบ similarity‑search (จากส่วน Methodology B)  
- **เลือกโมเดลภาษา**: เปรียบเทียบ Flan‑T5 รุ่นต่าง ๆ (XXL, BASE, SMALL) แล้วเลือก Flan‑T5‑XXL เนื่องจากให้ผลตอบสนองดีที่สุด (จากส่วน Methodology C)  
- **ผสาน LangChain**: สร้าง agent ที่ใช้ embeddings จาก FAISS และโมเดล Flan‑T5‑XXL เพื่อสร้างข้อความตอบกลับที่มีความต่อเนื่อง (จากส่วน Methodology C)  
- **นำเสนอผ่าน Gradio**: สร้างหน้าต่างแชทบนเว็บไซต์ด้วย Gradio เพื่อให้ผู้ใช้โต้ตอบกับ chatbot (จากส่วน Methodology D)  
- **ประเมินประสิทธิภาพ**: ทดสอบคำถามตัวอย่างและเปรียบเทียบคะแนนความพึงพอใจของผู้ใช้ระหว่างรุ่น XXL, BASE, SMALL (จากส่วน Results 

In [ ]:
response

In [ ]:
parameters = {"query": "langchain คืออะไร",
              "retriever": retriever,
              "llm": llm ,
               }

response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": "คำสำคัญในงานเขียนนี้มีอะไรบ้าง พร้อมคำอธิบายเป็นภาษาไทย",
              "retriever": retriever,
              "llm": llm ,
               }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": "บทที่ I. INTRODUCTION แปลความออกมาเป็นภาษาไทยทั้งหมด ",
              "retriever": retriever,
              "llm": llm ,
               }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response)

In [ ]:
display(response["answer"])

In [14]:
parameters = {"query": "II. LITERATURE SURVEY มีการพูดถึงหลักการ และทฤษฎีอะไรบ้าง สรุปออกมาหัวข้อทั้งหมด",
              "retriever": retriever,
              "llm": llm ,
               }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

**หัวข้อหลักที่กล่าวถึงในส่วน II. LITERATURE SURVEY**

- **การนำโมเดลใหญ่ (Large Neural Models) มาใช้ในงานค้นหาข้อมูล (Information Retrieval, IR)**  
  - ความท้าทายของการปรับใช้โมเดลทรานส์ฟอร์เมอร์ที่มีขนาดใหญ่ในระบบ IR พร้อมกับรักษาประสิทธิภาพ (จากงานของ S. Kim et al., 2023) (จากส่วน LITERATURE SURVEY)

- **การทำ Knowledge Distillation**  
  - การลดขนาดโมเดลผู้สอน (teacher) แบบ dual‑encoder และ cross‑encoder ลงเป็นโมเดลนักเรียน (student) ขนาด 1/10 ของต้นฉบับ โดยยังคงประสิทธิภาพ 95‑97 % (จากงานของ S. Kim et al., 2023)

- **การเปลี่ยนแปลงของสาขา IR ด้วยโมเดลทรานส์ฟอร์เมอร์ที่ฝึกล่วงหน้า (pre‑trained transformers)**  
  - การเกิดขึ้นของโมเดลใหญ่ทำให้เกิดการเปลี่ยนแปลงสำคัญใน IR (จากงานของ L. Bonifacio et al., 2022)

- **การใช้ชุดข้อมูล MS MARCO เพื่อ Zero‑Shot Transfer Learning**  
  - MS MARCO ช่วยให้โมเดลสามารถทำงานในหลาย ๆ งานได้โดยไม่ต้องฝึกใหม่ (จากงานของ L. Bonifacio et al., 2022)

- **แนวทางเปิด‑ซอร์ส (open‑source) ในการสร้าง LLM Chatbot จากข้อมูลเว็บไซต์**  
  - การใช้ความรู้เฉพา

In [15]:
parameters = {"query": "สรุป Intro หน่อยขอสั้น ๆ เข้าใจง่าย",
              "retriever": retriever,
              "llm": llm ,
               }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

**สรุปบทนำ (I. INTRODUCTION)**  

- ลูกค้าเป็นศูนย์กลางของธุรกิจและการบริการลูกค้าเป็นจุดต่อสู้หลักในการสร้างความไว้วางใจ (จากบทนำ)  
- การบริการลูกค้าในอดีตอาศัย FAQ และวิธีการสนับสนุนแบบดั้งเดิม แต่เทคโนโลยีใหม่ทำให้วิธีเหล่านี้เริ่มล้าสมัย (จากบทนำ)  
- การใช้โมเดลภาษาใหญ่ (LLMs) เช่น LangChain ช่วยให้บริการลูกค้าเป็นแบบตอบสนองแบบเรียลไทม์ มีความเป็นส่วนตัวและปรับตัวตามบริบท (จากบทนำ)  
- งานวิจัยเสนอ “Sahaay” – แพลตฟอร์มโอเพ่นซอร์สที่สามารถปรับขนาดได้สำหรับองค์กรทุกประเภท เพื่อให้บริการลูกค้าได้ทันที (จากบทนำ)  
- ตัวอย่างการนำเสนอใช้ข้อมูลเว็บไซต์ของ BVM Engineering College เพื่อแสดงให้เห็นว่าระบบสามารถดึงข้อมูลและตอบคำถามได้แบบเรียลไทม์ (จากบทนำ)


In [13]:
parameters = {"query": "ใน REFERENCES มีงานที่ถูกใช้อ้างอิงค์ในงานเขียนกี่งาน มีอะไรบ้างสรุปออกมาเป็นข้อ ๆ",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

**จำนวนงานอ้างอิง**  
- มีทั้งหมด **7 งาน** ที่ถูกอ้างอิงในส่วน REFERENCES

**รายการงานอ้างอิง (ตามลำดับที่ปรากฏในเอกสาร)**  

- **[1]** Asbjørn Følstad & Marita Skjuve (2019). *Chatbots for customer service: user experience and motivation*. Proceedings of the 1st International Conference on Conversational User Interfaces (CUI '19).  
- **[2]** Kim, S. et al. (2023). *EmbedDistill: A Geometric Knowledge Distillation for Information Retrieval*. ArXiv.  
- **[3]** Luiz Bonifacio, Hugo Abonizio, Marzieh Fadaee & Rodrigo Nogueira (2022). *InPars: Unsupervised Dataset Generation for Information Retrieval*. Proceedings of the 45th International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR '22).  
- **[4]** Su, Hongjin et al. (2022). *One Embedder, Any Task: Instruction-Finetuned Text Embeddings*. ArXiv.  
- **[5]** Johnson, Jeff, Matthijs Douze & Hervé Jégou (2017). *Billion-scale Similarity Search with GPUs*. ArXiv.  
- **[6]** Chung, Hyung W. et al. (202

In [ ]:
response

In [ ]:
parameters = {"query": "ฉันว่ามีมากกว่า 1 งานนะ",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": "Asbjørn Følstad and Marita Skjuve. 2019. Chatbots for customer service: user experience and motivation , อันนี้ไม่ใช่อ้างอิงหรอ",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])


In [ ]:
parameters = {"query": "แล้วส่วน REFERENCES นิยมใส่ข้อมูลอะไรเข้าไป",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": "ใน REFERENCES เอกสารที่เกี่ยวข้องในงานเขียนกี่งาน มีอะไรบ้างสรุปออกมาเป็นข้อ ๆ",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": """[1] Asbjørn Følstad and Marita Skjuve. 2019. Chatbots for customer service: user experience and motivation. In Proceedings of the 1st International Conference on Conversational User Interfaces (CUI '19). Association for Computing Machinery, New York, NY, USA, Article 1, 1–9. https://doi.org/10.1145/3342775.3342784 [2] Kim, S., Rawat, A. S., Zaheer, M., Jayasumana, S., Sadhanala, V., Jitkrittum, W., Menon, A. K., Fergus, R., & Kumar, S. (2023). EmbedDistill: A Geometric Knowledge Distillation for Information Retrieval. ArXiv. /abs/2301.12005 [3] Luiz Bonifacio, Hugo Abonizio, Marzieh Fadaee, and Rodrigo Nogueira. 2022. InPars: Unsupervised Dataset Generation for Information Retrieval. In Proceedings of the 45th International ACM SIGIR Conference on Research and Development in Information Retrieval (SIGIR '22). Association for Computing Machinery, New York, NY, USA, 2387–2392. https://doi.org/10.1145/3477495.3531863 [4] Su, Hongjin, Weijia Shi, Jungo Kasai, Yizhong Wang, Yushi Hu, Mari Ostendorf, Wen Yih, Noah A. Smith, Luke Zettlemoyer, and Tao Yu. "One Embedder, Any Task: Instruction-Finetuned Text Embeddings." ArXiv, (2022). /abs/2212.09741. [5] Johnson, Jeff, Matthijs Douze, and Hervé Jégou. "Billion-scale Similarity Search with GPUs." ArXiv, (2017). Accessed September 28, 2023. /abs/1702.08734. [6] Chung, Hyung W., Le Hou, Shayne Longpre, Barret Zoph, Yi Tay, William Fedus, Yunxuan Li et al. "Scaling Instruction-Finetuned Language Models." ArXiv, (2022). Accessed September 28, 2023. /abs/2210.11416 [7] Abid, A., Abdalla, A., Abid, A., Khan, D., Alfozan, A., & Zou, J. (2019). Gradio: Hassle-Free Sharing and Testing of ML Models in the Wild. ArXiv. /abs/1906.02569 
              แล้วพวกนี้คืออะไรถ้าไม่ใช่อ้างอิง""",
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_125" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])

In [ ]:
parameters = {"query": """ขอข้อมูล REFERENCES ทั้งหมดในรายงานนี้ 
              """,
              "retriever": retriever,
              "llm": llm ,
               }
kwargs = {"session_id":"user_129" }
response = RAG_answer_the_questions_history(**parameters, **kwargs)
print(response["answer"])